In [1]:
#Import The Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv("dataset_preprocess_ds.csv", index_col=None)

dataset = pd.get_dummies(dataset, drop_first=True)

In [4]:
dataset

,Crop/Year,Area_Hectares,Production_Tonnes,District_Name_BEED,District_Name_HINGOLI,District_Name_JALNA,District_Name_LATUR,District_Name_NANDED,District_Name_OSMANABAD,Winter/Summer_Rabi,...,Rice_Niger seed,Rice_Other Rabi pulses,Rice_Other Kharif pulses,Rice_Rice,Rice_Safflower,Rice_Sesamum,Rice_Soyabean,Rice_Sugarcane,Rice_Sunflower,Rice_Wheat
0,2014,32900,8700.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2014,45300,24200.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2014,101675,61475.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2014,4100,900.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2014,3700,1700.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2014,24100,7100.0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
147,2014,1900,800.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
148,2014,1300,NaN,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
149,2014,500,200.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
dataset.columns

Index(['Crop/Year', 'Area_Hectares', 'Production_Tonnes', 'District_Name_BEED',
       'District_Name_HINGOLI', 'District_Name_JALNA', 'District_Name_LATUR',
       'District_Name_NANDED', 'District_Name_OSMANABAD',
       'Winter/Summer_Rabi       ', 'Winter/Summer_Summer     ',
       'Winter/Summer_Whole Year ', 'Rice_Bajra', 'Rice_Castor seed',
       'Rice_Cotton(lint)', 'Rice_Gram', 'Rice_Groundnut', 'Rice_Jowar',
       'Rice_Linseed', 'Rice_Maize', 'Rice_Moong(Green Gram)',
       'Rice_Niger seed', 'Rice_Other  Rabi pulses',
       'Rice_Other Kharif pulses', 'Rice_Rice', 'Rice_Safflower',
       'Rice_Sesamum', 'Rice_Soyabean', 'Rice_Sugarcane', 'Rice_Sunflower',
       'Rice_Wheat'],
      dtype='object')

In [6]:
indep_X = dataset.drop(['Crop/Year', 'Area_Hectares', 'Production_Tonnes'], axis=1)
dep_Y = dataset['Rice_Soyabean']

In [7]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)

        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

In [8]:
def r2_prediction(regressor,X_test,y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2

In [9]:
def Linear(X_train,y_train,X_test):       
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 

In [10]:
def Decision(X_train,y_train,X_test):
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [11]:
def random(X_train,y_train,X_test):       
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [12]:
def xgboost(X_train,y_train,X_test):       
        from xgboost import XGBRegressor
        regressor = XGBRegressor(n_jobs=5,learning_rate=0.1,max_depth=10,random_state=1)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2

In [13]:
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    colnames_list = []  
    # List to store column names for each model
    r2_values = []  
    # List to store R2 values for each model

    from sklearn.linear_model import LinearRegression
    lin = LinearRegression()

    from sklearn.tree import DecisionTreeRegressor
    dec = DecisionTreeRegressor(random_state=0)

    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators=10, random_state=0)

    from xgboost import XGBRegressor
    xgb = XGBRegressor(n_jobs=5, learning_rate=0.1, max_depth=10, random_state=1)

    rfemodellist = [lin, dec, rf, xgb]

    for model in rfemodellist:
        log_rfe = RFE(estimator=model, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)

        # Get the column names selected by RFE
        selected_columns = [col for col, selected in zip(indep_X.columns, log_rfe.support_) if selected]
        colnames_list.append(selected_columns)

        # Fit the model and calculate and store the R2 value
        X_train, X_test, y_train, y_test = split_scalar(pd.DataFrame(log_rfe_feature), dep_Y)
        model.fit(X_train, y_train)  # Fit the model
        r2 = r2_prediction(model, X_test, y_test)
        r2_values.append(r2)

    return rfelist, colnames_list, r2_values

In [14]:
# Call the function with your data
rfelist, colnames_list, r2_values = rfeFeature(indep_X, dep_Y, 2)

In [15]:
# Print the selected column names and R2 values for each model
for model_name, selected_columns, r2_value in zip(["Linear", "Decision", "Random", "XGBoost"], colnames_list, r2_values):
    print(f"Model: {model_name}")
    print("Selected Columns:", selected_columns)
    print(f"R2 Value: {r2_value}\n")

Model: Linear
Selected Columns: ['Rice_Rice', 'Rice_Soyabean']
R2 Value: 1.0

Model: Decision
Selected Columns: ['Rice_Soyabean', 'Rice_Wheat']
R2 Value: 1.0

Model: Random
Selected Columns: ['Rice_Soyabean', 'Rice_Wheat']
R2 Value: 1.0

Model: XGBoost
Selected Columns: ['Rice_Soyabean', 'Rice_Wheat']
R2 Value: 0.9999997031172502



In [16]:
# Call the function with your data
rfelist, colnames_list, r2_values = rfeFeature(indep_X, dep_Y, 5)

In [17]:
# Print the selected column names and R2 values for each model
for model_name, selected_columns, r2_value in zip(["Linear", "Decision", "Random", "XGBoost"], colnames_list, r2_values):
    print(f"Model: {model_name}")
    print("Selected Columns:", selected_columns)
    print(f"R2 Value: {r2_value}\n")

Model: Linear
Selected Columns: ['Rice_Gram', 'Rice_Other Kharif pulses', 'Rice_Rice', 'Rice_Sesamum', 'Rice_Soyabean']
R2 Value: 1.0

Model: Decision
Selected Columns: ['Rice_Sesamum', 'Rice_Soyabean', 'Rice_Sugarcane', 'Rice_Sunflower', 'Rice_Wheat']
R2 Value: 1.0

Model: Random
Selected Columns: ['Rice_Sesamum', 'Rice_Soyabean', 'Rice_Sugarcane', 'Rice_Sunflower', 'Rice_Wheat']
R2 Value: 1.0

Model: XGBoost
Selected Columns: ['Rice_Sesamum', 'Rice_Soyabean', 'Rice_Sugarcane', 'Rice_Sunflower', 'Rice_Wheat']
R2 Value: 0.9999997031172502



In [18]:
#Model Creation

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import pickle

# Now you can use rfelist and colnames_list to train the Decision Tree model with the selected features
# Perform GridSearchCV for hyperparameter tuning
X_train, X_test, y_train, y_test = train_test_split(rfelist[1], dep_Y, test_size=0.25, random_state=0)

param_grid = {
    'criterion': ['gini', 'entropy', 'gini'],
    'splitter': ['best', 'random'],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train, y_train)
re=grid.cv_results_
y_predict=grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_predict)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test,y_predict)
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_predict,average='weighted')
print("The f1 macro value the best parameter{}:".format(grid.best_params_),f1_macro)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


The f1 macro value the best parameter{'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}: 1.0


C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\sreet\anaconda3\Lib\site-packages\

In [20]:
print(cm)

[[35  0]
 [ 0  3]]


In [21]:
print(clf_report )

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00         3

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0

In [23]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000477,0.000628,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
1,0.000867,0.000193,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
2,0.002597,0.001236,0.015822,0.010207,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.001751,0.000525,0.003528,0.000336,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.002007,0.000949,0.005767,0.004899,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
5,0.001635,0.000232,0.003731,0.000510,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
6,0.000482,0.000307,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
7,0.000605,0.000168,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
8,0.001035,0.001126,0.003271,0.002803,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
9,0.001947,0.000604,0.003230,0.001027,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [24]:
# Save the best model
filename = "Random_Forest_dataset_preprocess_ds_final.sav"
pickle.dump(grid, open(filename, 'wb'))

In [25]:
# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))

In [26]:
#Categorical Prediction

In [30]:
def get_categorical_prediction(prediction):
    return "Yes" if prediction == 1 else "No"

# Make predictions
Rice_Sesamum_input = float(input("Rice_Sesamum: "))
Rice_Soyabean_input = float(input("Rice_Soyabean: "))
Rice_Sugarcane_input = float(input("Rice_Sugarcane: "))
Rice_Sunflower_input = float(input("Rice_Sunflower: "))
Rice_Wheat_input = float(input("Rice_Wheat: "))
# Predict the future maintenance requirement
future_prediction = loaded_model.predict([[Rice_Sesamum_input,Rice_Soyabean_input,Rice_Sugarcane_input,Rice_Sunflower_input,Rice_Wheat_input]])

# Convert numerical prediction to categorical label
future_prediction_categorical = get_categorical_prediction(future_prediction[0])

Rice_Sesamum: 100
Rice_Soyabean: 75
Rice_Sugarcane: 100
Rice_Sunflower: 80
Rice_Wheat: 90


In [38]:
# Print the categorical prediction
print("Future Prediction (Rice_Sugarcane):", future_prediction_categorical)

Future Prediction (Rice_Sugarcane): No
